# IMDB 영화평 감정분석

In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv('../00.data/IMDB/labeledTrainData.tsv', header=0, sep ='\t', quoting=3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [4]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [5]:
# br태그를 공백으로 바꾸기
df['review'] = df.review.str.replace('<br />', ' ')

In [6]:
# 영어 이외의 문자는 공백으로 변환
import re

df['review'] = df.review.apply(lambda x : re.sub('[^a-zA-Z]', ' ',x))

In [7]:
from sklearn.model_selection import train_test_split

feature_df = df.drop(['id', 'sentiment'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(
    feature_df, df.sentiment, test_size = 0.3, random_state=156
)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

- CountVectorizer

In [9]:
count_vect = CountVectorizer(stop_words='english', ngram_range=(1,2))
count_vect.fit(X_train.review)
X_train_count = count_vect.transform(X_train.review)
X_test_count = count_vect.transform(X_test.review)

In [10]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_count, y_train)
pred = lr_clf.predict(X_test_count)
accuracy_score(y_test, pred)

0.886

- TfidfVectorizer

In [11]:
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tfidf_vect.fit(X_train.review)
X_train_tfidf = tfidf_vect.transform(X_train.review)
X_test_tfidf = tfidf_vect.transform(X_test.review)

In [12]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(X_train_tfidf, y_train)
pred = lr_clf.predict(X_test_tfidf)
accuracy_score(y_test, pred)

0.8936

### 모델 저장하고 불러오기

In [13]:
import joblib
joblib.dump(tfidf_vect, 'model/imdb_vect.pkl')
joblib.dump(lr_clf, 'model/imdb_lr.pkl')

['model/imdb_lr.pkl']

In [14]:
# 지우기
del tfidf_vect
del lr_clf

In [15]:
new_vect = joblib.load('model/imdb_vect.pkl')
new_lr = joblib.load('model/imdb_lr.pkl')

In [16]:
new_X_test = new_vect.transform(X_test.review)

In [17]:
pred = new_lr.predict(new_X_test)
accuracy_score(y_test, pred)

0.8936

### Pipeline을 써서 학습/예측/평가

In [21]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('count_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])
pipeline.fit(X_train.review, y_train)
pred = pipeline.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logistic Regression 정확도 : {acc:.4f}')

Count Vectorizer + Logistic Regression 정확도 : 0.8860


In [22]:
joblib.dump(pipeline, 'model/pipeline.pkl')

['model/pipeline.pkl']

In [23]:
new_pipe = joblib.load('model/pipeline.pkl')

In [25]:
pred = new_pipe.predict(X_test.review)
acc = accuracy_score(y_test, pred)
print(f'Count Vectorizer + Logistic Regression 정확도 : {acc:.4f}')

Count Vectorizer + Logistic Regression 정확도 : 0.8860


In [63]:
pipeline2 = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

In [64]:
pipeline2.get_params()

{'memory': None,
 'steps': [('tfidf_vect', TfidfVectorizer(stop_words='english')),
  ('lr_clf', LogisticRegression())],
 'verbose': False,
 'tfidf_vect': TfidfVectorizer(stop_words='english'),
 'lr_clf': LogisticRegression(),
 'tfidf_vect__analyzer': 'word',
 'tfidf_vect__binary': False,
 'tfidf_vect__decode_error': 'strict',
 'tfidf_vect__dtype': numpy.float64,
 'tfidf_vect__encoding': 'utf-8',
 'tfidf_vect__input': 'content',
 'tfidf_vect__lowercase': True,
 'tfidf_vect__max_df': 1.0,
 'tfidf_vect__max_features': None,
 'tfidf_vect__min_df': 1,
 'tfidf_vect__ngram_range': (1, 1),
 'tfidf_vect__norm': 'l2',
 'tfidf_vect__preprocessor': None,
 'tfidf_vect__smooth_idf': True,
 'tfidf_vect__stop_words': 'english',
 'tfidf_vect__strip_accents': None,
 'tfidf_vect__sublinear_tf': False,
 'tfidf_vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf_vect__tokenizer': None,
 'tfidf_vect__use_idf': True,
 'tfidf_vect__vocabulary': None,
 'lr_clf__C': 1.0,
 'lr_clf__class_weight': None,
 'lr_clf__

In [65]:
params = {
    'tfidf_vect__ngram_range' : [(1,1), (1,2), (1,3)],
    'tfidf_vect__max_df'  : [300, 500, 700],
    'lr_clf__C': [1, 10]
}

In [66]:
from sklearn.model_selection import GridSearchCV
grid_pipe = GridSearchCV(pipeline2, param_grid= params, cv=3, scoring='accuracy', verbose=1)

grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 13.0min finished
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.8822287469759523


In [67]:
pred = grid_pipe.predict(X_test.review)
accuracy_score(y_test, pred)

0.8873333333333333

In [68]:
params = {
    'tfidf_vect__ngram_range' : [(1,2), (1,3)],
    'tfidf_vect__max_df'  : [600,700,800],
    'lr_clf__C': [9, 10, 11]
}

In [69]:
grid_pipe = GridSearchCV(pipeline2, param_grid= params, cv=3, scoring='accuracy', verbose=1)

grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed: 22.3min finished
{'lr_clf__C': 9, 'tfidf_vect__max_df': 800, 'tfidf_vect__ngram_range': (1, 2)} 0.8856573184269131


In [70]:
pred = grid_pipe.predict(X_test.review)
accuracy_score(y_test, pred)

0.8869333333333334

In [76]:
params = {
    'tfidf_vect__ngram_range' : [(1,2)],
    'tfidf_vect__max_df'  : [750,800,850],
    'lr_clf__C': [7, 8, 9]
}

In [77]:
grid_pipe = GridSearchCV(pipeline2, param_grid= params, cv=3, scoring='accuracy', verbose=1)

grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  6.3min finished
{'lr_clf__C': 8, 'tfidf_vect__max_df': 800, 'tfidf_vect__ngram_range': (1, 2)} 0.8857144645495487


In [78]:
pred = grid_pipe.predict(X_test.review)
accuracy_score(y_test, pred)

0.8864

In [80]:
params = {
    'tfidf_vect__ngram_range' : [(1,2)],
    'tfidf_vect__max_df'  : [800],
    'lr_clf__C': [8],
    'tfidf_vect__min_df': [1,2,3]
}

In [81]:
grid_pipe = GridSearchCV(pipeline2, param_grid= params, cv=3, scoring='accuracy', verbose=1)

grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.3min finished
{'lr_clf__C': 8, 'tfidf_vect__max_df': 800, 'tfidf_vect__min_df': 2, 'tfidf_vect__ngram_range': (1, 2)} 0.8862287600625516


In [82]:
pred = grid_pipe.predict(X_test.review)
accuracy_score(y_test, pred)

0.8918666666666667

In [93]:
params = {
    'tfidf_vect__ngram_range' : [(1,2)],
    'tfidf_vect__max_df'  : [800, 900, 1000],
    'lr_clf__C': [8,10,20,30],
    'tfidf_vect__min_df': [2],
}

In [94]:
grid_pipe = GridSearchCV(pipeline2, param_grid= params, cv=3, scoring='accuracy', verbose=1)

grid_pipe.fit(X_train.review, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  3.9min finished
{'lr_clf__C': 30, 'tfidf_vect__max_df': 900, 'tfidf_vect__min_df': 2, 'tfidf_vect__ngram_range': (1, 2)} 0.887543022929583


In [95]:
pred = grid_pipe.predict(X_test.review)
accuracy_score(y_test, pred)

0.8925333333333333